In [ ]:
import numpy as np
from google.colab import drive
import pandas as pd
import json

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
species_true_only_df3 = pd.read_csv("/content/drive/MyDrive/Project/species_true_only_table3.csv")

species_true_only_df3.head()

,Unnamed: 0,SourceFile,Subject,DateTimeOriginal,PathCorrect
0,1,D:/NetworkRail22/Barnes_Tagged/17_NR_11.04.202...,"contact1, placeID: 17, species1: Fox",2022:03:26 20:40:35,True
1,2,D:/NetworkRail22/Barnes_Tagged/17_NR_11.04.202...,"contact1, placeID: 17, species1: Fox",2022:03:27 19:23:14,True
2,3,D:/NetworkRail22/Barnes_Tagged/17_NR_11.04.202...,"contact1, placeID: 17, species1: Fox",2022:03:28 19:25:16,True
3,4,D:/NetworkRail22/Barnes_Tagged/17_NR_11.04.202...,"contact1, placeID: 17, species1: Fox",2022:03:29 01:13:38,True
4,5,D:/NetworkRail22/Barnes_Tagged/17_NR_11.04.202...,"placeID: 17, species1: Fox",2022:03:29 01:13:41,True


In [ ]:
double_species_count = 0

for i in range(len(species_true_only_df3)):
  subject = str(species_true_only_df3.iloc[i]['Subject'])
  if 'species2' in subject:
    double_species_count += 1

print(f'{double_species_count = }')

double_species_count = 11651


In [ ]:
import pandas as pd
import re
import numpy as np

# Initialize lists to store data for the new DataFrame
place_ids = []
species = []
source_files = []
dates = []
times = []

# Loop through the DataFrame
for index, row in species_true_only_df3.iterrows():
    subject = row['Subject']

    if pd.isna(subject):  # Check for NaN values
        place_id = ''
        species_name = ''
    else:
        # Extracting PlaceID
        place_id_match = re.search(r'placeID: (\d+)', subject)
        if place_id_match:
            place_id = place_id_match.group(1)
        else:
            place_id = ''

        # Check if 'species2' is present
        if 'species2' in subject:
            species_name = ''  # If 'species2' is present, leave 'Species' empty
        else:
            # Extracting Species
            species_match = re.search(r'species1: ([^,]+)', subject)
            if species_match:
                species_name = species_match.group(1)
            else:
                species_name = ''

        # Extract date and time from 'DateTimeOriginal' column
        datetime_original = row['DateTimeOriginal']
        if pd.notna(datetime_original):  # Check for NaN values
            date_match = re.search(r'(\d+:\d+:\d+) (\d+:\d+:\d+)', datetime_original)  # Assuming 'DateTimeOriginal' has 'YYYY:MM:DD HH:MM:SS' format
            if date_match:
                date = date_match.group(1)
                time = date_match.group(2)
            else:
                date = ''
                time = ''
        else:
            date = ''
            time = ''

    # Append to lists
    place_ids.append(place_id)
    species.append(species_name)
    source_files.append(row['SourceFile'])
    dates.append(date)
    times.append(time)

# Create a new DataFrame
new_df = pd.DataFrame({
    'SourceFile': source_files,
    'PlaceID': place_ids,
    'Subject': species,
    'Date': dates,
    'Time': times
})

# Print the new DataFrame
new_df.head()


,SourceFile,PlaceID,Subject,Date,Time
0,D:/NetworkRail22/Barnes_Tagged/17_NR_11.04.202...,17,Fox,2022:03:26,20:40:35
1,D:/NetworkRail22/Barnes_Tagged/17_NR_11.04.202...,17,Fox,2022:03:27,19:23:14
2,D:/NetworkRail22/Barnes_Tagged/17_NR_11.04.202...,17,Fox,2022:03:28,19:25:16
3,D:/NetworkRail22/Barnes_Tagged/17_NR_11.04.202...,17,Fox,2022:03:29,01:13:38
4,D:/NetworkRail22/Barnes_Tagged/17_NR_11.04.202...,17,Fox,2022:03:29,01:13:41


In [ ]:
# Define a function to clean the species name
def clean_species_name(species_name):
    return species_name.lower().replace("_", "").replace(" ", "")

# Clean the 'Subject' column in-place
new_df['Subject'] = new_df['Subject'].apply(clean_species_name)

# Print the modified DataFrame
new_df.head()


,SourceFile,PlaceID,Subject,Date,Time
0,D:/NetworkRail22/Barnes_Tagged/17_NR_11.04.202...,17,fox,2022:03:26,20:40:35
1,D:/NetworkRail22/Barnes_Tagged/17_NR_11.04.202...,17,fox,2022:03:27,19:23:14
2,D:/NetworkRail22/Barnes_Tagged/17_NR_11.04.202...,17,fox,2022:03:28,19:25:16
3,D:/NetworkRail22/Barnes_Tagged/17_NR_11.04.202...,17,fox,2022:03:29,01:13:38
4,D:/NetworkRail22/Barnes_Tagged/17_NR_11.04.202...,17,fox,2022:03:29,01:13:41


In [2]:
# Initialize a dictionary to store unique entries and their counts
subject_counts = {}

for subject in new_df['Subject']:
    if isinstance(subject, str) and subject.strip():  # Check if the subject is a non-empty string
        if subject not in subject_counts:
            subject_counts[subject] = 1
        else:
            subject_counts[subject] += 1

# Print the unique entries and their counts
for subject, count in subject_counts.items():
    print(f"{subject}: {count}")


NameError: name 'new_df' is not defined

In [1]:
import matplotlib.pyplot as plt

subject_counts_filtered = {k: v for k, v in subject_counts.items() if v >= 50}
subject_counts_filtered = {k: v for k, v in sorted(subject_counts_filtered.items(), key=lambda item: item[1])}
# Plotting the bar chart with log scale on y-axis
plt.figure(figsize=(10, 6))
plt.bar(subject_counts_filtered.keys(), subject_counts_filtered.values(), color='skyblue')
plt.yscale('log')  # Set y-axis scale to log
plt.xlabel('Subject')
plt.ylabel('Count (log scale)')
plt.title('Image Count per Species (Count >= 50)')
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Project/species_count.png')
plt.show()

NameError: name 'subject_counts' is not defined

In [ ]:
new_df.to_csv('/content/drive/MyDrive/Project/data_for_MD.csv', index=False)